In [1]:
import os
import torch
import torch.nn as nn
import numpy as np

In [10]:
from torchvision import models

In [11]:
from models.seresnet.FastPose import createModel

In [15]:
from src.opt import opt

In [16]:
opt.loadModel = "exp/ceiling_sparse_duc_se/5e-6/5e-6_best_auc.pkl"

In [17]:
model =  createModel(cfg=None).cpu()

In [18]:
model.load_state_dict(torch.load(weights, map_location="cpu"))

<All keys matched successfully>

In [19]:
def obtain_prune_idx(path):
    lines = []
    with open(path, 'r') as f:
        file = f.readlines()
        for line in file:
            lines.append(line)
            
    idx = 0
    prune_idx = []
    for line in lines:
        if "):" in line:
            idx  += 1
        if "BatchNorm2d" in line:
            #print(idx, line)
            prune_idx.append(idx)
    
    prune_idx = prune_idx[1:] # 去除第一个bn1层
    return prune_idx

model_name = "./model.txt"
print(model, file=open(model_name, 'w'))
prune_idx = obtain_prune_idx(model_name)

In [20]:
len(prune_idx)

105

In [21]:
def sort_bn(model, prune_idx):
    size_list = [m.weight.data.shape[0] for idx, m in enumerate(model.modules()) if idx in prune_idx]
    # bn_layer = [m for m in model.modules() if isinstance(m, nn.BatchNorm2d)]
    bn_prune_layers = [m for idx, m in enumerate(model.modules()) if idx in prune_idx]
    bn_weights = torch.zeros(sum(size_list))

    index = 0
    for module, size in zip(bn_prune_layers, size_list):
        bn_weights[index:(index + size)] = module.weight.data.abs().clone()
        index += size
    sorted_bn = torch.sort(bn_weights)[0]
    
    return sorted_bn

sorted_bn = sort_bn(model, prune_idx)

In [22]:
sorted_bn

tensor([3.0307e-09, 6.0363e-09, 4.6829e-08,  ..., 2.5850e-01, 2.8974e-01,
        3.3852e-01])

In [23]:
def obtain_bn_mask(bn_module, thre):
    if device != "cpu":
        thre = thre.cuda()
    mask = bn_module.weight.data.abs().ge(thre).float()

    return mask

def obtain_filters_mask(model, prune_idx, thre):
    pruned = 0
    bn_count = 0
    total = 0
    num_filters = []
    pruned_filters = []
    filters_mask = []
    pruned_maskers = []
    
    for idx, module in enumerate(model.modules()):
        if isinstance(module, nn.BatchNorm2d):
            if idx in prune_idx:
                mask = obtain_bn_mask(module, thre).cpu().numpy()
                remain = int(mask.sum())
                pruned = pruned + mask.shape[0] - remain

                if remain == 0: # 保证至少有一个channel
                    # print("Channels would be all pruned!")
                    # raise Exception
                    max_value = module.weight.data.abs().max()
                    mask = obtain_bn_mask(module, max_value).cpu().numpy()
                    remain = int(mask.sum())
                    pruned = pruned + mask.shape[0] - remain
                    bn_count += 1
                print(f'layer index: {idx:>3d} \t total channel: {mask.shape[0]:>4d} \t '
                      f'remaining channel: {remain:>4d}')
                
                pruned_filters.append(remain)
                pruned_maskers.append(mask.copy())
            else:
                mask = np.ones(module.weight.data.shape)
                remain = mask.shape[0]
            
            total += mask.shape[0]
            num_filters.append(remain)
            filters_mask.append(mask.copy())
    
    prune_ratio = pruned / total
    print(f'Prune channels: {pruned}\tPrune ratio: {prune_ratio:.3f}')

    return pruned_filters, pruned_maskers

In [24]:
percent = 0.8

In [26]:
def obtain_bn_threshold(model, sorted_bn, percentage):
    thre_index = int(len(sorted_bn) * percentage)
    thre = sorted_bn[thre_index]
    
    return thre

threshold = obtain_bn_threshold(model, sorted_bn, percent)
print(threshold)

tensor(0.0002)


In [27]:
device = "cpu"

In [28]:
pruned_filters, pruned_maskers = obtain_filters_mask(model, prune_idx, threshold)

layer index:   9 	 total channel:   64 	 remaining channel:   23
layer index:  11 	 total channel:   64 	 remaining channel:   27
layer index:  13 	 total channel:  256 	 remaining channel:  101
layer index:  23 	 total channel:  256 	 remaining channel:  121
layer index:  26 	 total channel:   64 	 remaining channel:    8
layer index:  28 	 total channel:   64 	 remaining channel:   16
layer index:  30 	 total channel:  256 	 remaining channel:  111
layer index:  33 	 total channel:   64 	 remaining channel:   18
layer index:  35 	 total channel:   64 	 remaining channel:   11
layer index:  37 	 total channel:  256 	 remaining channel:   94
layer index:  41 	 total channel:  128 	 remaining channel:   55
layer index:  43 	 total channel:  128 	 remaining channel:   91
layer index:  45 	 total channel:  512 	 remaining channel:  223
layer index:  55 	 total channel:  512 	 remaining channel:  260
layer index:  58 	 total channel:  128 	 remaining channel:   52
layer index:  60 	 total 

In [29]:
print(pruned_filters, file=open("ceiling.txt", "w"))